In [3]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import PorterStemmer
import ast

ModuleNotFoundError: No module named 'numpy'

In [ ]:
# Read CSV files
credits_df = pd.read_csv("credits.csv")
movies_df = pd.read_csv("movies.csv")

In [ ]:
# Merge datasets on movie ID
movies_df = movies_df.merge(credits_df, left_on='id', right_on='movie_id', how='left')

In [ ]:
# Select relevant features
movies_df = movies_df[['movie_id', 'title_x', 'genres', 'keywords', 'overview', 'cast', 'crew']]
movies_df.columns = ['movie_id', 'title', 'genres', 'keywords', 'overview', 'cast', 'crew']

In [ ]:
# Define a function to convert JSON-like string to a list of dictionary items
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L


In [ ]:
# Apply the convert function to genres and keywords columns
movies_df['genres'] = movies_df['genres'].apply(convert)
movies_df['keywords'] = movies_df['keywords'].apply(convert)

In [ ]:
# Extract the names of the top 3 cast members
def get_cast(obj):
    L = []
    count = 0
    for i in ast.literal_eval(obj):
        if count < 3:
            L.append(i['name'])
        else:
            break
        count += 1
    return L
    
movies_df['cast'] = movies_df['cast'].apply(get_cast)

In [ ]:
# Extract the director's name from the crew column
def get_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

movies_df['crew'] = movies_df['crew'].apply(get_director)

In [ ]:
# Define a function to remove spaces and convert to lowercase
def collapse(L):
    return ' '.join(L).replace(" ", "").lower()

movies_df['genres'] = movies_df['genres'].apply(collapse)
movies_df['keywords'] = movies_df['keywords'].apply(collapse)
movies_df['cast'] = movies_df['cast'].apply(collapse)
movies_df['crew'] = movies_df['crew'].apply(collapse)

In [ ]:
# Combine all features into a single string
movies_df['tags'] = movies_df['genres'] + ' ' + movies_df['keywords'] + ' ' + movies_df['cast'] + ' ' + movies_df['crew'] + ' ' + movies_df['overview']

In [ ]:
movies_df["tags"].isna().sum()

3

In [ ]:
movies_df.dropna(inplace = True)

In [ ]:
# Create a CountVectorizer object
cv = CountVectorizer(max_features=5000, stop_words='english')
vector = cv.fit_transform(movies_df['tags']).toarray()

In [ ]:
# Compute the cosine similarity matrix
similarity = cosine_similarity(vector)

In [ ]:
# Define a function to recommend movies based on the title
def recommend(movie):
    movie_index = movies_df[movies_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    
    for i in movie_list:
        print(movies_df.iloc[i[0]].title)

In [ ]:
recommend('The Dark Knight')

The Dark Knight Rises
Batman Forever
Batman Returns
Batman Begins
Batman: The Dark Knight Returns, Part 2


In [ ]:
recommend('Avatar')

Apollo 18
Beowulf
Tears of the Sun
The American
Hanna


In [ ]:
recommend('Batman')

The Dark Knight Rises
Batman & Robin
Batman Begins
Enemy at the Gates
Batman Forever


In [ ]:
recommend('Man of Steel')

Ong Bak 2
War Horse
Beginners
Nicholas Nickleby
The Slaughter Rule


In [ ]:
recommend('Spider-Man')

Kick-Ass
The Amazing Spider-Man 2
The Amazing Spider-Man
Spider-Man 3
The New Guy


In [ ]:
recommend('Titanic')

The Switch
Romance & Cigarettes
Captain Phillips
Ghost Ship
The Notebook


In [ ]:
recommend('Inception')

Up
Peaceful Warrior
Pitch Perfect 2
Hotel Transylvania 2
Partition
